In [3]:
# !pip install opencv-python
# !pip install ultralytics opencv-python pillow

In [1]:
import os
image_path = "/Users/sajin/ML/Projects/Agri-1/Fly/Images/"  # replace with your trap image
processed_path = "/Users/sajin/ML/Projects/Agri-1/Fly/Images/processed/"
images_list = os.listdir(image_path)
images_list = [image_path+x for x in images_list if x.find('det') ==-1 & x.find('preprocessed_') == -1 & x.find('processed') == -1 ]
print(images_list)

['/Users/sajin/ML/Projects/Agri-1/Fly/Images/IMG-20250817-WA0081.jpg', '/Users/sajin/ML/Projects/Agri-1/Fly/Images/IMG-20250817-WA0082.jpg', '/Users/sajin/ML/Projects/Agri-1/Fly/Images/IMG-20250817-WA0083.jpg', '/Users/sajin/ML/Projects/Agri-1/Fly/Images/IMG-20250817-WA0084.jpg', '/Users/sajin/ML/Projects/Agri-1/Fly/Images/IMG-20250817-WA0085.jpg']


In [2]:
images_list[3]

'/Users/sajin/ML/Projects/Agri-1/Fly/Images/IMG-20250817-WA0084.jpg'

In [ ]:
from huggingface_hub import login
login("")

In [7]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "mps"
print(f"Running on {DEVICE}")

# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-500M-Instruct")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    dtype=torch.bfloat16,
    _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

# # Create input messages
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image"},
#             {"type": "text", "text": "Can you describe this image?"}
#         ]
#     },
# ]

# # Prepare inputs
# prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image], return_tensors="pt")
# inputs = inputs.to(DEVICE)

# # Generate outputs
# generated_ids = model.generate(**inputs, max_new_tokens=500)
# generated_texts = processor.batch_decode(
#     generated_ids,
#     skip_special_tokens=True,
# )

# print(generated_texts[0])
"""
Assistant: The image depicts a large, historic statue of liberty, located in New York City. The statue is a green, cylindrical structure with a human figure at the top, holding a torch. The statue is situated on a pedestal that resembles the statue of liberty, which is located on a small island in the middle of a body of water. The water surrounding the island is calm, reflecting the blue sky and the statue.
In the background, there are several tall buildings, including the Empire State Building, which is visible in the distance. These buildings are made of glass and steel, and they are positioned in a grid-like pattern, giving them a modern look. The sky is clear, with a few clouds visible, indicating fair weather.
The statue is surrounded by trees, which are green and appear to be healthy. There are also some small structures, possibly houses or buildings, visible in the distance. The overall scene suggests a peaceful and serene environment, typical of a cityscape.
The image is taken during the daytime, likely during the day of the statue's installation. The lighting is bright, casting a strong shadow on the statue and the water, which enhances the visibility of the statue and the surrounding environment.
To summarize, the image captures a significant historical statue of liberty, situated on a small island in the middle of a body of water, surrounded by trees and buildings. The sky is clear, with a few clouds visible, indicating fair weather. The statue is green and cylindrical, with a human figure holding a torch, and is surrounded by trees, indicating a peaceful and well-maintained environment. The overall scene is one of tranquility and historical significance.
"""


Running on mps


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

/Users/sajin/miniforge3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:2244: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


"\nAssistant: The image depicts a large, historic statue of liberty, located in New York City. The statue is a green, cylindrical structure with a human figure at the top, holding a torch. The statue is situated on a pedestal that resembles the statue of liberty, which is located on a small island in the middle of a body of water. The water surrounding the island is calm, reflecting the blue sky and the statue.\nIn the background, there are several tall buildings, including the Empire State Building, which is visible in the distance. These buildings are made of glass and steel, and they are positioned in a grid-like pattern, giving them a modern look. The sky is clear, with a few clouds visible, indicating fair weather.\nThe statue is surrounded by trees, which are green and appear to be healthy. There are also some small structures, possibly houses or buildings, visible in the distance. The overall scene suggests a peaceful and serene environment, typical of a cityscape.\nThe image is

In [9]:
# Create input messages

# Load images
image = load_image(images_list[2])

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "You are an expert entomologist specializing in agricultural pest monitoring. \
Analyze the following image of a yellow sticky trap used in crop fields.\
Your task is to:\
1. Detect all insects or pests visible in the image.\
2. For each insect, provide:\
   - The most likely pest name (e.g., aphid, whitefly, thrips, spider mite, fungus gnat, leafhopper, beetle).\
   - A bounding box in the format [x_min, y_min, x_max, y_max] relative to the image size.\
   - A confidence score between 0 and 1.\
3. If unsure, label it as \"unknown insect\" with low confidence. "}
        ]
    },
]

# Prepare inputs
N=1
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image], size={"longest_edge": N*512}, return_tensors="pt")
inputs = inputs.to(DEVICE)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])

User:You are an expert entomologist specializing in agricultural pest monitoring. Analyze the following image of a yellow sticky trap used in crop fields.Your task is to:1. Detect all insects or pests visible in the image.2. For each insect, provide:   - The most likely pest name (e.g., aphid, whitefly, thrips, spider mite, fungus gnat, leafhopper, beetle).   - A bounding box in the format [x_min, y_min, x_max, y_max] relative to the image size.   - A confidence score between 0 and 1.3. If unsure, label it as "unknown insect" with low confidence. 
Assistant: The image shows a yellow sticky trap used in crop fields. The trap is designed to trap and immobilize insects, which are typically pests in agriculture. The trap is made of a sticky material that is sticky enough to stick to the insect's body, and it is designed to be water-resistant. The trap is mounted on a pole, and it is attached to a metal rod.

The trap is filled with water, and the water is sprayed onto the insect. The insec